In [16]:
import os
from pathlib import Path
import pandas as pd
from tabulate import tabulate
from pprint import pprint

In [9]:
os.environ["DATA_DIR"] = str(Path().cwd().parent.parent / "data")

In [12]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 10
pd.set_option("display.max_colwidth", None)

In [3]:
targets = [
    "United States",
    "United Kingdom",
    "Canada",
    "Australia",
    "Germany",
    "Japan",
    "Netherlands",
]

In [13]:
output_dir = Path().cwd().parent / "output"


def read_data(path: Path) -> pd.DataFrame:
    return pd.read_json(path)


for i in Path(os.getenv("DATA_DIR")).rglob("*.json"):
    print(f"tabulating...: {i}")
    df = read_data(i)
    df = df[df["country"].isin(targets)].sort_values("country")
    (output_dir / f"{i.stem}.txt").write_text(
        tabulate(df, headers="keys", tablefmt="grid", showindex=False)
    )

tabulating...: /workspaces/sign-to-migrate/data/informal_employment.json


tabulating...: /workspaces/sign-to-migrate/data/annual_growth_rate_per_worker.json
tabulating...: /workspaces/sign-to-migrate/data/social_protection.json
tabulating...: /workspaces/sign-to-migrate/data/happiness.json
tabulating...: /workspaces/sign-to-migrate/data/cpi.json
tabulating...: /workspaces/sign-to-migrate/data/unemployment_rate_disability.json
tabulating...: /workspaces/sign-to-migrate/data/unemployment_rate.json
tabulating...: /workspaces/sign-to-migrate/data/women_in_senior_and_middle_position.json
tabulating...: /workspaces/sign-to-migrate/data/average_hourly_earnings.json
tabulating...: /workspaces/sign-to-migrate/data/youth_neet_proportion.json
tabulating...: /workspaces/sign-to-migrate/data/labour_rights.json
tabulating...: /workspaces/sign-to-migrate/data/women_in_managerial_position.json
tabulating...: /workspaces/sign-to-migrate/data/working_poverty_rate.json
tabulating...: /workspaces/sign-to-migrate/data/cost_of_living.json


In [31]:
df = read_data(Path(f"{os.getenv('DATA_DIR')}/happiness.json"))
display(df.head())
df[
    [
        "country",
        "Happiness.Rank",
        "Happiness.Score",
        "Economy..GDP.per.Capita.",
        "Family",
        "Health..Life.Expectancy.",
        "Freedom",
        "Generosity",
        "Trust..Government.Corruption.",
        "Dystopia.Residual",
    ]
]

,Country,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual,year,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Health (Life Expectancy),Trust (Government Corruption),Dystopia Residual,Overall rank,country,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Perceptions of corruption,Lower Confidence Interval,Upper Confidence Interval
0,Norway,1.0,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027,2017,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Denmark,2.0,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707,2017,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Iceland,3.0,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715,2017,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Switzerland,4.0,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716,2017,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Finland,5.0,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182,2017,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,country,Happiness.Rank,Happiness.Score,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
0,None,1.0,7.537,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,None,2.0,7.522,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,None,3.0,7.504,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,None,4.0,7.494,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,None,5.0,7.469,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182
...,...,...,...,...,...,...,...,...,...,...
777,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,0.083000,NaN,NaN
778,Tanzania,NaN,NaN,NaN,NaN,NaN,NaN,0.270000,NaN,NaN
779,South Sudan,NaN,NaN,NaN,NaN,NaN,NaN,0.224000,NaN,NaN
780,Central African Republic,NaN,NaN,NaN,NaN,NaN,NaN,0.218000,NaN,NaN
